In [1]:
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
import requests
import json
import unicodedata
from difflib import SequenceMatcher

In [ ]:
def similarity_ratio(str1, str2):
    """Calculates the similarity ratio between two strings."""
    return SequenceMatcher(None, str1, str2).ratio()

def find_book_google_books_advanced(title, author, title_similarity_threshold=0.8, author_similarity_threshold=0.8):
    """
    Searches the Google Books API for a book using advanced matching logic.

    Args:
        title (str): The title of the book.
        author (str): The author of the book.
        title_similarity_threshold (float): Minimum similarity ratio for title match (0.0 to 1.0).
        author_similarity_threshold (float): Minimum similarity ratio for author match (0.0 to 1.0).

    Returns:
        dict or None: A dictionary containing information about the best matching book,
                     or None if no sufficiently similar book is found or an error occurs.
    """
    base_url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": f"{title} inauthor:{author}"}
    best_match = None
    highest_similarity = 0.0

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        if items:
            for item in items:
                info = item.get("volumeInfo", {})
                title_api = info.get("title", "").lower()
                authors_api = [a.lower() for a in info.get("authors", [])]

                title_similarity = similarity_ratio(title.lower(), title_api)
                author_similarity = 0.0
                if authors_api:
                    max_author_similarity = 0.0
                    for author_from_api in authors_api:
                        current_similarity = similarity_ratio(author.lower(), author_from_api)
                        max_author_similarity = max(max_author_similarity, current_similarity)
                    author_similarity = max_author_similarity
                elif not author:  # If no author is provided in the search, consider it a match
                    author_similarity = 1.0

                combined_similarity = (title_similarity + author_similarity) / 2.0

                if title_similarity >= title_similarity_threshold and author_similarity >= author_similarity_threshold:
                    if combined_similarity > highest_similarity:
                        highest_similarity = combined_similarity
                        best_match = info

        # If no good match with both title and author, try searching by title only with a lower threshold
        if not best_match:
            params_title_only = {"q": title}
            response_title_only = requests.get(base_url, params=params_title_only)
            response_title_only.raise_for_status()
            data_title_only = response_title_only.json()
            items_title_only = data_title_only.get("items", [])

            if items_title_only:
                for item in items_title_only:
                    info = item.get("volumeInfo", {})
                    title_api = info.get("title", "").lower()
                    title_similarity = similarity_ratio(title.lower(), title_api)
                    if title_similarity >= title_similarity_threshold - 0.1: # Lower threshold for title-only search
                        if title_similarity > highest_similarity:
                            highest_similarity = title_similarity
                            best_match = info

        return best_match

    except requests.exceptions.RequestException as e:
        print(f"Error during API request: {e}")
        return None
    
def add_columns(sheet, num_columns_to_add):
    """Adds a specified number of columns to the Google Sheet."""
    try:
        properties = sheet.fetch_sheet_metadata()['sheets'][0]['properties']
        current_column_count = properties['gridProperties']['columnCount']
        new_column_count = current_column_count + num_columns_to_add

        body = {
            "requests": [
                {
                    "updateSheetProperties": {
                        "properties": {
                            "gridProperties": {
                                "columnCount": new_column_count
                            }
                        },
                        "fields": "gridProperties.columnCount"
                    }
                }
            ]
        }
        sheet.batch_update(body)

    except gspread.exceptions.APIError as e:
        print(f"Error adding columns: {e}")

def insert_info_in_sheet(sheet, info_to_add):
    """
    Inserts information into the Google Sheets based on the provided parameters.

    Args:
        sheet (gspread.models.Spreadsheet): The Google Sheets object.
        info_to_add (str): The field name (e.g. 'Category') to be fetched and added to the sheet.

    Returns:
        None
    """
    sheet1 = sheet.sheet1
    values_list = sheet1.get_all_values()

    info_searched = info_to_add.lower()
    header_row = values_list[0]
    num_cols = len(header_row)

    # Get column indices for title and author
    title_column_index = header_row.index("Book_Title") if "Book_Title" in header_row else 0
    author_column_index = header_row.index("Author") if "Author" in header_row else 1

    # Add new column if info_to_add is not in header
    if info_searched.capitalize() not in header_row:
        sheet1.add_cols(1)
        num_cols += 1
        sheet1.update_cell(1, num_cols, info_searched.capitalize())
        info_column_index = num_cols - 1
    else:
        info_column_index = header_row.index(info_searched.capitalize())

    # Get rows where info_to_add is empty or missing
    empty_rows = [
        i + 2 for i, row in enumerate(values_list[1:])  # skip header
        if len(row) <= info_column_index or not row[info_column_index]
    ]

    try:
        for row_index in empty_rows:
            if row_index - 1 < len(values_list):
                book = values_list[row_index - 1]
                if len(book) <= max(title_column_index, author_column_index):
                    continue  # skip incomplete rows

                title_name = book[title_column_index].strip()
                author_name = book[author_column_index].strip()

                book_api_info = find_book_google_books_advanced(title_name, author_name)

                if book_api_info and info_to_add in book_api_info:
                    info_value = book_api_info[info_to_add]
                    if not info_value:
                        info_value = "Unknown"
                else:
                    info_value = "Unknown"

                print(f"Updating row {row_index}: '{title_name}' → {info_searched.capitalize()} = {info_value}")
                # Convert list or other non-string types to string
                if isinstance(info_value, list):
                    info_value = ", ".join(info_value)
                elif not isinstance(info_value, str):
                    info_value = str(info_value)

                sheet1.update_cell(row_index, info_column_index + 1, info_value)


    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API Error: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_book_keys(sheet):
    """
    Efficiently retrieves the set of keys returned by the Google Books API for each unique book.
    Avoids redundant API calls and skips header row properly.

    Args:
        sheet (gspread.models.Spreadsheet): The Google Sheets object.

    Returns:
        list: A list of unique sets of keys from the book API results.
    """
    values_list = sheet.sheet1.get_all_values()
    header_row = values_list[0]

    try:
        title_col = header_row.index("Book_Title")
    except ValueError:
        print("Warning: 'Book_Title' column not found. Assuming index 0.")
        title_col = 0

    try:
        author_col = header_row.index("Author")
    except ValueError:
        print("Warning: 'Author' column not found. Assuming index 1.")
        author_col = 1

    keys = []
    seen_titles = set()

    for row in values_list[1:]:  # skip header
        if len(row) <= max(title_col, author_col):
            continue  # skip malformed rows

        title = row[title_col].strip()
        author = row[author_col].strip()

        key = (title.lower(), author.lower())
        if key in seen_titles:
            continue  # skip duplicates

        seen_titles.add(key)

        book_info = find_book_google_books_advanced(title, author)
        if book_info:
            api_keys_set = frozenset(book_info.keys())
            if api_keys_set not in keys:
                keys.append(api_keys_set)

    return keys

def save_sheet_as_sql(sheet, table_name="books", output_file="sheet_export.sql"):
    """
    Saves the contents of a Google Sheet as SQL INSERT statements in a .sql file.

    Args:
        sheet (gspread.models.Spreadsheet): Authenticated Google Sheet object.
        table_name (str): Name of the SQL table.
        output_file (str): Path to the output .sql file.
    """
    sheet1 = sheet.sheet1
    values = sheet1.get_all_values()

    headers = values[0]  # column names
    rows = values[1:]    # data rows

    with open(output_file, "w", encoding="utf-8") as f:
        for row in rows:
            # Pad row to match header length
            row += [""] * (len(headers) - len(row))

            # Sanitize and format values
            sanitized_values = []
            for cell in row:
                # Handle strings and escape single quotes
                if isinstance(cell, str):
                    cell = cell.replace("'", "''")
                    sanitized_values.append(f"'{cell}'")
                else:
                    sanitized_values.append("NULL" if cell == "" else str(cell))

            insert_stmt = f"INSERT INTO {table_name} ({', '.join(headers)}) VALUES ({', '.join(sanitized_values)});"
            f.write(insert_stmt + "\n")

    print(f"SQL export complete: {output_file}")

In [3]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
gc = gspread.authorize(creds)

In [4]:
sheet_id = "19JV7gS3Iy_MTOluozSech3iFmkxRF11oCVYLhp1ziN0"
sheet = gc.open_by_key(sheet_id)

values_list = sheet.sheet1.get_all_values()
records = sheet.sheet1.get_all_records()  # fast and efficient

In [5]:
pd.DataFrame(values_list, columns=values_list[0]).drop(index=0)

,Book_Title,Author,Publisher,Number_of_pages,Translator,Year_bought,Status,Owner,Categories,Language,Publisheddate
1,O Assassinato No Expresso Oriente,Agatha Christie,Harper Colins,196,Archibaldo Figueira,2017,Lido,Diego,Fiction,pt-BR,2014-04-15
2,Morte no Nilo,Agatha Christie,Harper Colins,248,Newton Goldman,2017,Lido,Diego,Fiction,pt-BR,2015-04-17
3,Um corpo na Biblioteca,Agatha Christie,Harper Colins,180,Edilson Alkimin Cunha,2017,Lido,Diego,Fiction,pt-BR,2015-01-15
4,Admirável mundo novo,Aldous Huxley,Biblioteca Azul,306,Vidal de Oliveira,2022,Lido,Diego,Fiction,pt-BR,2014-03-06
5,Nietzsche para estressados,Allan Percy,Editora Ática,128,-,-,Lido,Solange,Self-Help,pt-BR,2011-10-17
...,...,...,...,...,...,...,...,...,...,...,...
106,A arte da Guerra,Sun Tzu,Jardim dos Livros,125,André da Silva Bueno,2010,Lido,Diego,Literary Collections,pt-BR,2014-02-05
107,Metrópolis,Thea Von Harbou,Aleph,458,Petê Rissatti,2020,Lido,Diego,Fiction,en,2021-03-23
108,Depois daquela viagem,Valéria Piassa Polizzi,Editora Ática,277,-,-,Lido,Diego,Young Adult Fiction,pt-BR,2013
109,Neuromancer,William Gibson,Aleph,319,Fábio Fernandes,2019,Lido,Diego,Fiction,pt-BR,2015-09-16


In [6]:
infos_to_search = get_book_keys(sheet)
print(f"Infos to search: {infos_to_search}")

Infos to search: [frozenset({'authors', 'imageLinks', 'language', 'contentVersion', 'allowAnonLogging', 'maturityRating', 'description', 'canonicalVolumeLink', 'title', 'panelizationSummary', 'publisher', 'previewLink', 'ratingsCount', 'publishedDate', 'infoLink', 'printType', 'readingModes', 'categories', 'industryIdentifiers', 'pageCount', 'averageRating'}), frozenset({'authors', 'imageLinks', 'language', 'contentVersion', 'allowAnonLogging', 'maturityRating', 'description', 'canonicalVolumeLink', 'title', 'panelizationSummary', 'publisher', 'previewLink', 'publishedDate', 'infoLink', 'printType', 'readingModes', 'categories', 'industryIdentifiers', 'pageCount'}), frozenset({'authors', 'imageLinks', 'language', 'contentVersion', 'allowAnonLogging', 'subtitle', 'maturityRating', 'description', 'canonicalVolumeLink', 'title', 'panelizationSummary', 'publisher', 'previewLink', 'ratingsCount', 'publishedDate', 'infoLink', 'printType', 'readingModes', 'categories', 'industryIdentifiers', 

In [7]:
info = "categories"

insert_info_in_sheet(sheet, info)

In [8]:
info = "language"

insert_info_in_sheet(sheet, info)

In [9]:
info = "publishedDate"

insert_info_in_sheet(sheet, info)

In [11]:
save_sheet_as_sql(sheet, table_name="books", output_file="MyLibrary.sql")

SQL export complete: MyLibrary.sql


___